In [ ]:
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from pylib.tf_utils import mnist_train, mnist_test
from tensorflow.keras.utils import to_categorical as one_hot

import matplotlib.pyplot as plt

from pylib.draw_nn import draw_neural_net_fig   

X_train, y_train = mnist_train()
X_test, y_test = mnist_test()

y_train = one_hot(y_train)
y_test  = one_hot(y_test)

N_PIXELS= 28 * 28
N_CLASSES = 10

<!-- requirement: pylib/__init__.py -->
<!-- requirement: pylib/draw_nn.py -->
<!-- requirement: pylib/tf_utils.py -->
<!-- requirement: pylib/mnist_dataset.py -->
<!-- requirement: images/dnn_overfit.png -->
<!-- requirement: images/dnn_reg.png -->
<!-- requirement: images/dnn_dropout.png -->
<!-- requirement: images/dnn_batch.png -->
<!-- requirement: small_data/cal_house.json.gz -->

# Optimization

Deep neural networks are complex models containing many model parameters. This makes training deep neural networks more challenging than other models. Deep neural networks often require large data sets to learn from, which may require special handling for sufficiently speedy processing. Also it can be difficult to get neural networks to converge on the optimal set of model parameters, either due to sluggish searching of parameter space or due to overfitting. In this notebook we will discuss monitoring neural network learning and how we might improve our models and their learning.

## Stochastic Gradient Descent

In most of our examples we have used the full training set in training our models.  With a small number of observations, this is easy enough.  But when dealing with millions of observations (or more), this will become infeasible. It's also unnecessary.  Most observations in a large data set will have many neighbors that impart almost the same information.  We don't need all of them in our learning process.

If we take this reasoning to the extreme, we can train based on the error of a single observation at a time.  We want to train with the "most unexpected" datum each time.  In lieu of trying to figure this out, we'll feed the data randomly. This process is known as **stochastic gradient descent**.  Each individual step will be noisy, since it contains only a small amount of information.  This means that you will need more steps and a smaller learning rate.

In addition to generally being faster than batch processing, stochastic gradient descent is often more robust.  The noise inherent in this process is often enough to push us out of local minima of the loss function.  (A downside of this noise it that it makes it harder for the model to finally converge at the end.)  Additionally, stochastic gradient descent supports **online learning**.  We can update an existing model with new data just by running some descent steps on it.

This approach introduces the "batch size" hyperparameter into our learning regimen, the size of the random sample at each gradient descent step. We'll examine the effect of the batch size on gradient descent and its relationship with learning rate using a linear model, but the ideas extend naturally to more complex models.

In [ ]:
import gzip
import json
from sklearn.model_selection import ShuffleSplit

with gzip.open("small_data/cal_house.json.gz", "r") as fin:
    housing = json.load(fin)
    
for train, test in ShuffleSplit(1, 0.2, random_state=42).split(housing['data']):
    X_train = np.array(housing['data'])[train].astype(np.float32)
    y_train = np.array(housing['target'])[train].astype(np.float32)
    X_test = np.array(housing['data'])[test].astype(np.float32)
    y_test = np.array(housing['target'])[test].astype(np.float32)
    
X_mean = X_train.mean(axis=0)
X_std = X_train.std(axis=0)

Xs_train = (X_train - X_mean) / X_std
Xs_test = (X_test - X_mean) / X_std

In [ ]:
from tqdm import tqdm

class AutoLinearRegression():
    def __init__(self, eta=.1):
        self.W = tf.Variable(0.)
        self.b = tf.Variable(0.)
        self.opt = tf.keras.optimizers.SGD(learning_rate=eta)
    
    def loss(self, X, y, return_func=False):
        def loss_():
            return tf.reduce_mean(tf.square(X * self.W + self.b - y))
        
        if not return_func:
            return loss_()
        
        return loss_
    
    def fit(self, X, y, steps=10):
        for _ in range(steps):
            self.opt.minimize(self.loss(X, y, return_func=True), [self.W, self.b])
            
        return self

In [ ]:
BATCH_SIZE = 10
ETAS = [.2, .1, .01]

errors = []
Ws = []

for e in ETAS:
    model = AutoLinearRegression(eta=e)
    errors.append([model.loss(Xs_test[:, 0:1], y_test.reshape(-1, 1)).numpy()])
    Ws.append([model.W.numpy()])
    for i in tqdm(range(len(y_train) // BATCH_SIZE)):
        j = np.random.choice(len(y_train), BATCH_SIZE, replace=False)
        model.fit(Xs_train[j, 0:1], y_train[j].reshape(-1, 1), steps=1)
        errors[-1].append(model.loss(Xs_test[:, 0:1], y_test[:].reshape(-1, 1)).numpy())
        Ws[-1].append(model.W.numpy())

In [ ]:
plt.plot(np.array(errors).T, '.-')
plt.legend(ETAS, title=r'$\eta$')
plt.xlabel('step')
plt.ylabel('loss');
plt.ylim((0, np.array(errors)[:, 0].max() * 1.1));

### Exercise: Hyperparameter Tuning

Adjust the learning rate and the batch size.  How quickly can you get the model to converge?

## Exploring the loss surface and learning curves

For linear regression on a single feature, the loss function is simple enough to visualize.

In [ ]:
EPOCHS = 10
BATCH_SIZE = len(y_train)
ETAS = [0.1, 0.5, 0.8, 1.01]

errors = []
Ws = []

for e in ETAS:
    model = AutoLinearRegression(eta=e)
    errors.append([model.loss(Xs_test[:, 0:1], y_test.reshape(-1, 1)).numpy()])
    Ws.append([model.W.numpy()])
    for _ in range(EPOCHS):
        for i in range(len(y_train) // BATCH_SIZE):
            j = np.random.choice(len(y_train), BATCH_SIZE, replace=False)
            model.fit(Xs_train[j, 0:1], y_train[j].reshape(-1, 1), steps=1)
            errors[-1].append(model.loss(Xs_test[:, 0:1], y_test[:].reshape(-1, 1)).numpy())
            Ws[-1].append(model.W.numpy())

plt.plot(np.array(Ws).T, np.array(errors).T, '.-')
# This is just an eyeball estimate of the loss function
plt.plot(np.linspace(-0.4, 2.0),
         7.8 * (np.linspace(-0.4, 2.0) - 0.79)**2 + 0.6, 'k:')
plt.legend(ETAS, title=r'$\eta$')
plt.xlabel('$W$')
plt.ylabel('loss');

We can see small learning rates may converge slowly, but large learning rates are prone to overshooting minima and may even lead to model divergence.

For a complex model, the loss function is a function of many model parameters, and visualizing it is impossible. The loss surface may have many valleys and plateaus where our model may get stuck as we search the landscape looking for the deepest valley, but the general trade-off between speed of convergence and danger of divergence holds (with the additional complication that very small learning rates may make models more prone to getting trapped in local minima).

> The learning rate may be chosen by trial and error, but it is usually best to choose it by monitoring learning curves that plot the objective function as a function of time.  This is more of an art than a science, and most guidance on this subject should be regarded with some skepticism.

> &mdash;Ian Goodfellow, Yoshua Bengio, and Aaron Courville, [Deep Learning](http://www.deeplearningbook.org/)

If you get confused by the behavior of your learning curves, don't despair.  You are in [good company](http://lossfunctions.tumblr.com/).


## Overfitting

*N.B. It takes longer to properly train the models in this section than can easily be accommodated within the length of time it takes to give this lecture.  If you wish to work with the fully trained models, please adjust the cell below and be aware that each model will take several minutes to train.* 

In [ ]:
n_epochs = 1 # 25

The more complex a model, the greater the danger of **overfitting**.  Overfitting occurs when the model becomes so powerful that it can fit not only the general signal but also the specific noise present in your training data.

Let's start with a simple contrived example.  We'll make up a set of data (blue dots) and fit it with two curves, a linear model (green) and a $19^{th}$-degree polynomial model (red).  The results are below:

In [ ]:
x = np.linspace(-1, 1, 20).reshape(-1, 1)
xl = np.linspace(-1, 1, 200).reshape(-1, 1)
X = np.column_stack(x**n for n in range(1,20))
XL = np.column_stack(xl**n for n in range(1,20))

np.random.seed(1)
y = 2 * x + np.random.normal(0, .2, (20, 1))

m, _, _, _ = np.linalg.lstsq(x, y, rcond=None)
W, _, _, _ = np.linalg.lstsq(X, y, rcond=None)

plt.plot(x, y, '.', ms=15)
plt.plot(xl, xl.dot(m))
plt.plot(xl, XL.dot(W))
plt.ylim(-2.3, 2.3)

print("Linear Model MSE (train):", np.mean((y - x.dot(m))**2) / len(y))
print("Polynomial Model MSE (train):", np.mean((y - X.dot(W))**2) / len(y))

Which model is the better model?  If you simply look at the mean-squared error, it's the polynomial model.  It's error is less than a tenth of that of the linear model, and we see that it goes almost right through all the data points.

But most of us would look at this result and agree that the linear model is more likely.  There's nothing to suggest that those large excursions at either extreme is actually real.  Our intuition tells us that it's more likely we have a simple underlying signal with some noise than a complicated underlying signal.

Ultimately, the only way to tell for sure is to test with additional data.  Therefore, it's standard practice to separate our data into training and test, or validation, sets.  These sets should share the same target signal, but they will have different noise.

Since we're just making up data for the example, we'll make up a test data set here as well.

In [ ]:
xt = np.random.uniform(-1, 1, 11).reshape(-1, 1)
XT = np.column_stack(xt**n for n in range(1,20))

yt = 2 * xt + np.random.normal(0, .2, (11, 1))

plt.plot(x, y, '.', ms=15, alpha=0.2)
plt.plot(xl, xl.dot(m))
plt.plot(xl, XL.dot(W))
plt.plot(xt, yt, '.', ms=15)
plt.ylim(-2.3, 2.3)

print("Linear Model MSE (test):", np.mean((yt - xt.dot(m))**2) / len(yt))
print("Polynomial Model MSE (test):", np.mean((yt - XT.dot(W))**2) / len(yt))

In this case, we see that the linear model behave comparably on the training and test sets, which tells us it is not overfitting.  The polynomial model, on the other hand, does terribly worse on the test set, telling us that it is very much overfit.

This should not surprise us.  The polynomial model had 19 degrees of freedom and only 20 points to go through.  Of course it could get near to all of them!

Now consider that neural net.  The hidden layer alone as `N_PIXELS` $\times$ `64` weights acting as degrees of freedom.  Including the weights in the output layer and the biases, there are 50,890 degrees of freedom in this model.  Our training set has only 55,000 observations in it, so it might seem that overfitting should be a huge problem.  While there is some evidence of overfitting in the split between the training and test sets, it isn't so large.

In fact, neural networks have a surprising ability to resist overfitting, given their flexibility.  This resistance is not absolute, though; given enough freedom, a neural net will eventually overfit.  There are countermeasures.  To demonstrate them, we'll first build a larger network that will overfit more easily.  This will have two hidden layers of 1024 neurons each.

In [ ]:
hidden_size = 1024

model = keras.models.Sequential()

model.add(
    keras.layers.Dense(
        hidden_size,
        activation='sigmoid',
        kernel_initializer=keras.initializers.TruncatedNormal(stddev=N_PIXELS**-0.5)
    )
)
model.add(
    keras.layers.Dense(
        hidden_size, 
        activation='sigmoid',
        kernel_initializer=keras.initializers.TruncatedNormal(stddev=hidden_size**-0.5)
    )
)
model.add(
    keras.layers.Dense(
        N_CLASSES,
        activation='softmax',
        kernel_initializer=keras.initializers.TruncatedNormal(stddev=hidden_size**-0.5)
    )
)

model.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.SGD(lr=0.3), 
              metrics=['accuracy'])                 
                                                       
history_overfit = model.fit(X_train, y_train,
                            epochs=n_epochs,
                            batch_size=30,
                            validation_data=(X_test,y_test))

If you trained this model over tens of thousands of steps, you would find that this neural net has managed to completely learn the training set: it's getting 100% accuracy on the training data consistently.  But on the test data, it struggles to get past 98% accuracy.

The situation is even more stark when the loss function is considered.  The training loss is approaching zero, suggesting that there is nothing left to be learned.  The testing loss, however, reaches a minimum and then starts increasing.  This is a clear sign of overfitting.  Past this point, the neural network is learning primarily about the noise in the training set, which will hurt its performance on the test set.

If we allowed our model to train for 50,000 steps, the loss of our model on our testing and training steps would evolve to look something like this:

<table>
    <tr>
        <td> <img src="files/images/dnn_overfit.png" style="width: 600px;"/> </td>
    </tr>
</table>

This suggests a simple approach to dealing with overfitting: Stop the process when the test performance starts declining.  This technique is known as **early stopping**.  In practice you don't want to stop as soon as performance gets worse once.  You will (probably) notice steps where the performance decreased briefly before improving again.  The usual approach is to save the current state every time it beats the previous best model.  If we manage to go a certain number of steps without beating the best model, we'll stop and restore the state from that model.

You can implement early stopping in Keras using [callbacks](https://keras.io/callbacks/).

Early stopping ensures that we get the best model we noticed and keeps us from doing counterproductive training.  Other techniques we will discuss can yield better absolute performance, but they too can benefit from early stopping.

## Regularization

Overfitting is caused by the model having too much flexibility, and the main source of flexibility in a neural network is the weights in the neurons.  Specifically, non-zero weights indicate some relationship between the input and the output.  **Regularization** penalizes this flexibility by penalizing non-zero weights.  Thus, the network will only have a weight be non-zero if the benefit to the loss function is greater than the penalty applied for the weight.

There are two main types of regularization: $L_2$-regularization adds a penalty proportional to the sum of the squares of the weights, while $L_1$-regularization uses the sum of the absolute values of the weights.  (The biases are generally not regularized.)

$$ L_2\text{ term: }\alpha\sum W^2 \qquad L_1\text{ term: }\alpha\sum\left|W\right| $$

The hyperparameter $\alpha$ is the regularization parameter.  Its size needs to be set to provide the right amount of flexibility that the net avoids both overfitting and its converse, underfitting.  In general, you will need to do a bit of a search to find the appropriate value for your problem.

> **Aside:** Regularization actually has a Bayesian motivation.  In our simple example with the polynomial fit, we realized that we had a prior: we thought a linear model with noise was more probable than a $19^{th}$-degree polynomial model.  Similarly, in the case of neural networks we expect weights of zero are more probable than non-zero weights.  The regularization terms add in the Bayesian prior that we expect the weights to come from a certain distribution centered about zero.

The `keras.layers.Dense` function takes a `kernel_regularizer` argument.  A regularizer can be passed in, and Keras will add a new term to the loss function containing the regularization penalty for that layer.

In the code below, we set up a neural network in the same way we did before, with the addition of a `keras.regularizers.l2` on each level.  This takes the parameter $\alpha$ as an argument.  We found this value of alpha to work after some trial and error.

In [ ]:
from tensorflow.keras import regularizers

alpha = 2.5 * 10 ** -7

model = keras.models.Sequential()

model.add(
    keras.layers.Dense(
        hidden_size, 
        activation='sigmoid',
        kernel_initializer=keras.initializers.TruncatedNormal(stddev=N_PIXELS**-0.5),
        kernel_regularizer=regularizers.l2(alpha)
    )
)
model.add(
    keras.layers.Dense(
        hidden_size, 
        activation='sigmoid',
        kernel_initializer=keras.initializers.TruncatedNormal(stddev=hidden_size**-0.5),
        kernel_regularizer=regularizers.l2(alpha)
    )
)
model.add(
    keras.layers.Dense(
        N_CLASSES,
        activation='softmax',
        kernel_initializer=keras.initializers.TruncatedNormal(stddev=hidden_size**-0.5),
        kernel_regularizer=regularizers.l2(alpha)
    )
)

model.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.SGD(lr=0.3), 
              metrics=['accuracy'])

history_reg = model.fit(X_train, y_train,
                        epochs=n_epochs,
                        batch_size=30,
                        validation_data=(X_test,y_test))

The following graph illustrates how the loss of the original and regularized networks evolve over 50,000 training steps

<table>
    <tr>
        <td> <img src="files/images/dnn_reg.png" style="width: 600px;"/> </td>
    </tr>
</table>

Note that the gap between training and test performance is smaller, indicating the overfitting has been significantly diminished.  This is effected by both an improvement in the test performance and a decrease in the training performance.  The test performance plateaus, but does not seem to start growing.

## Dropout

Regularization is a popular mechanism for simple linear models, where training is rather fast.  This makes it easy to try a number of values for $\alpha$ in order to determine the optimal value.  Deep nets take much longer to train, making this search much more painful.

For this reason, another approach for dealing with overfitting has been widely adopted.  Introduced by Hinton, Srivastava, *et al*, in two [recent](https://arxiv.org/pdf/1207.0580.pdf) [papers](http://jmlr.org/papers/volume15/srivastava14a/srivastava14a.pdf), **dropout** is a remarkably simple strategy.  At each training step, a random selection of neurons are removed from the network:

In [ ]:
# Network with 50% dropput
draw_neural_net_fig([20, 14, 12, 10], 0.5)

The idea behind this is to keep individual neurons from becoming too specialized.  Because each neuron may or may not be in any given run, it cannot be the only neuron detecting a particular feature.  If that feature is important, responsibility for its detection needs to be spread out among several neurons.  When we make a prediction, we will keep all of the neurons, in order to make the best prediction possible.

Another way of thinking about dropout is that, instead of training a single network of $N$ neurons, we are training $2^N$ networks of different combinations of neurons.  Then when we make a prediction, we are essentially taking an ensemble average of all of these $2^N$ networks.  A well-known feature of ensemble models is that they tend to avoid overfitting.

Dropout is provided by the `keras.layers.Dropout` function.  In addition to setting some fraction $p$ of the components of the input tensor to zero, it scales the remaining components by $1/(1-p)$.  This ensures that that average input to each layer will be the same during training and testing.  Keras will automatically disable dropout during testing and prediction.

The standard value for dropout is 50%, which works well for most problems.  In this example, we'll use only a 20% dropout.  It trains a bit faster, but it doesn't do quite as good of a job of dealing with overfitting.

In [ ]:
rate = 0.2 

model = keras.models.Sequential()

model.add(keras.layers.Dropout(rate))
model.add(
    keras.layers.Dense(
        hidden_size, 
        activation='sigmoid',
        kernel_initializer=keras.initializers.TruncatedNormal(stddev=N_PIXELS**-0.5)
    )
)
model.add(keras.layers.Dropout(rate))
model.add(
    keras.layers.Dense(
        hidden_size, 
        activation='sigmoid',
        kernel_initializer=keras.initializers.TruncatedNormal(stddev=hidden_size**-0.5)
    )
)
model.add(keras.layers.Dropout(rate))
model.add(
    keras.layers.Dense(
        N_CLASSES,
        activation='softmax',
        kernel_initializer=keras.initializers.TruncatedNormal(stddev=hidden_size**-0.5)
    )
)

model.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.SGD(lr=0.3),
              metrics=['accuracy'])            
                                               
history_drop = model.fit(X_train, y_train,            
                         epochs=n_epochs,                  
                         batch_size=30,
                         validation_data=(X_test,y_test))

Note that dropout contributes to the training scores reported above but not to the test scores. If we want a fair comparison between training and test performance (to measure overfitting), then we need to write a custom loop. The following code demonstrates the syntax (although in practice we may save computation by reporting metrics less often than once per epoch).

In [ ]:
for _ in range(3):
    model.fit(X_train, y_train,
              epochs=1,
              batch_size=30,
              verbose=0)
    print("Train:", model.evaluate(X_train, y_train))
    print("Test:", model.evaluate(X_test, y_test))       

<table>
    <tr>
        <td> <img src="files/images/dnn_dropout.png" style="width: 600px;"/> </td>
    </tr>
</table>

We see that incorporating dropout into our network produced better performance than both the original and regularized neural networks.

## Batch Normalization

Another [recently-developed](https://arxiv.org/pdf/1502.03167v3.pdf) tool for deep networks is **batch normalization**.  Although it can help with overfitting, it was originally developed to deal with the vanishing gradient problem.  Recall that activation functions have flat regions, where their gradients are small.  When the input is in these regions, gradient descent will only move the weights small amounts, leaving them stuck in the low-gradient regions.  Intelligent choices for initializations and activation functions try to avoid this as much as possible.

Batch normalization takes a more proactive approach, scaling and shifting the inputs so that the average input, over the whole batch, has a target mean and standard deviation.  These target values become parameters of the model, tuned during training.

By keeping gradients from vanishing, batch normalization reduces the importance of the weight initialization and the activation function.  Larger learning rates can be used.  Although the initial steps may proceed more slowly, as the correct normalizations must be learned, learning should proceed much faster overall than without.  Batch normalization can also have a regularization effect, reducing the propensity towards overfitting!

The `keras.layers.BatchNormalization` function takes care of all of this work for us.  Because normalization should be applied directly before the activation, we can no longer use the activation option built into `keras.layers.Dense`.  Instead, we apply the activation function by hand.  This implementation uses an exponentially-weighted moving average to calculate the batch means and standard deviations.  This is governed by the momentum parameter, which specifies the weight of the previous estimate.  It should usually be close to 1, even more so for large data sets and small batches.

In [ ]:
m = 0.99

model = keras.models.Sequential()

model.add(
    keras.layers.Dense(
        hidden_size, 
        activation=None,
        kernel_initializer=keras.initializers.TruncatedNormal(stddev=N_PIXELS**-0.5)
    )
)
model.add(keras.layers.BatchNormalization(momentum=m))
model.add(keras.layers.Activation('sigmoid'))

model.add(
    keras.layers.Dense(
        hidden_size, 
        activation=None,
        kernel_initializer=keras.initializers.TruncatedNormal(stddev=hidden_size**-0.5)
    )
)
model.add(keras.layers.BatchNormalization(momentum=m))
model.add(keras.layers.Activation('sigmoid'))

model.add(
    keras.layers.Dense(
        N_CLASSES,
        activation='softmax',
        kernel_initializer=keras.initializers.TruncatedNormal(stddev=hidden_size**-0.5)
    )
)
       
model.compile(loss='categorical_crossentropy',             
              optimizer=keras.optimizers.SGD(lr=0.05), 
              metrics=['accuracy'])            
                                        
history_batch = model.fit(X_train, y_train,             
                          epochs=n_epochs,                  
                          batch_size=30,
                          validation_data=(X_test,y_test))

<table>
    <tr>
        <td> <img src="files/images/dnn_batch.png" style="width: 600px;"/> </td>
    </tr>
</table>

We can see that training with batch normalization proceeded more quickly.  It appears the the normalization effect is missing here; the final performance is very similar to the basic net performance.

### Exercise: Implement early stopping


Add early stopping to the optimization routine by adding an `EarlyStopping` [callback](https://keras.io/callbacks/). Pay special attention to the `patience` and `restore_best_weights` arguments.

### Exercise: Explore dropout


Change the dropout rate of the model.  (The usual standard is 50%.)  How does it impact the eventual performance of the model? The amount of overfitting?  The time it takes to converge?

*Copyright &copy; 2019 Pragmatic Institute. This content is licensed solely for personal use. Redistribution or publication of this material is strictly prohibited.*